# Fast simulation event display and perfromance parameterization
Expert  configurable event display for fast simulzation studies  


## Contents 
[Import modules](#import) <br>
[Load trees, set aliases](#load) <br>
[XYZ display pt](#XYZdisplayPt) <br>
[XYZ display pt](#XYZdisplaytrack) <br>

In [ ]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
from ROOT import TFile, gSystem
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter, RandomForest
import plotly.express as px
import time
import pickle
from IPython.display import Image, clear_output
from RootInteractive.InteractiveDrawing.bokeh.CDSCompress import *
#
from fastSimulation import *

## 1.) Load trees and define derived variables:
<a id='load'></a>

* Covaraince matrix of Alice kalman track - stored as an array 
* ```(y,z,sing*phi),tgl(theta),qPt) -> (0,2,5,9,14)```
 * 0   
 * 1 2
 * 3 4 5 
 * 6 7 8 9
 * 10 11 12 13 14 

In [ ]:
inputData = "/home2/miranov/github/fastMCKalman/data/alice3/fastParticle.list";
inputData = "${fastMCKalman}/data/alice3/fastParticle.list";
inputData = "/home2/miranov/github/fastMCKalman/data/testTPC/fastParticle.list";
#inputData = "/home2/miranov/github/fastMCKalman/data/run2/fastParticle.list";

In [ ]:
ROOT.initTreeFast(inputData)
tree=ROOT.treeFast

In [ ]:
tree.SetMarkerStyle(25)
tree.SetMarkerSize(0.5)
ROOT.setAliases(tree)
tree.SetAlias("qPt","part.fParamMC[0].fP[4]")
tree.SetAlias("ptMC","part.fParamMC[0].fData.Pt()")
tree.SetAlias("ptIn","part.fParamIn[1].fData.Pt()")
tree.SetAlias("ptMCP","part.fParamMC[].fData.Pt()")
tree.SetAlias("tglMC","part.fParamMC[0].fP[3]")
tree.SetAlias("tglIn","part.fParamIn[1].fP[3]")
#
tree.SetAlias("sigmaY0","sqrt(part.fParamIn[1].fC[0])")
tree.SetAlias("sigmaZ0","sqrt(part.fParamIn[1].fC[2])")
tree.SetAlias("sigmaqPt0","sqrt(part.fParamIn[1].fC[14])")
#
tree.SetAlias("sigmaY0Rot","sqrt(part.fParamInRot[1].fC[0])")
tree.SetAlias("sigmaZ0Rot","sqrt(part.fParamInRot[1].fC[2])")
tree.SetAlias("sigmaqPt0Rot","sqrt(part.fParamInRot[1].fC[14])")
#
tree.SetAlias("gzMC","(part.fParamMC[].fP[1])")
tree.SetAlias("gzIn","(part.fParamIn[].fP[1])")
tree.SetAlias("rMC","sqrt(gxMC**2+gyMC**2)")
tree.SetAlias("snpMC","(part.fParamMC[].fP[2])")
tree.SetAlias("pIndex","Iteration$")
tree.SetAlias("trackID","i")
tree.SetAlias("snpMax","abs(part.fParamMC[fMaxLayer].fP[2])")

## Root tree - fast query check

In [ ]:
c1=ROOT.TCanvas("c1","c1",700,400)
entries = tree.Draw("sigmaY0:1/ptMC:pidCode","ptMC>0.1&&abs(tglMC)<2&&!isSecondary&&sigmaY0<1&&fMaxLayer>3","colz")
#c1.SetLogy(1)
c1.Draw("colz")

In [ ]:
c1=ROOT.TCanvas("c1","c1",700,400)
entries = tree.Draw("10*sigmaY0Rot:1/ptMC:abs(tglMC)","ptMC>0.3&&abs(tglMC)<2&&!isSecondary&&sigmaY0Rot<1&&isSecondary==0&&sigmaY0Rot<0.1&&fMaxLayer>3","colz")
#c1.SetLogy(1)
c1.Draw("colz")

In [ ]:
tree.GetAlias("gzMC")
anyTree=treeToAnyTree(tree)
output_notebook()

## Resolution plot - outliers in covariance matrix could be too small
* resolution at layer 0 as obtained by covariance matrix 
* problem similar in 2 rotation schema
  * rotate to local after each propagation
  * only one rotation

In [ ]:
defaultCutTrack="ptMC>0.01&&sigmaY0<0.2&&sigmaZ0<0.2&&sigmaY0Rot<0.2&&sigmaZ0Rot<0.2&&fMaxLayer>3"
dfResDraw=tree2Panda(tree,["qPt","tglMC",
                           "sigmaY0","sigmaZ0","sigmaqPt0",
                           "sigmaY0Rot","sigmaZ0Rot","sigmaqPt0Rot",
                           "pidCode","isSecondary","fMaxLayer","dEdxExp",
                          "snpMax"],
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=5000)

output_file("fastSimulDisplayReolution.html")
dfDraw=dfResDraw.head(400000)
figureArray = [
    [['qPt'], ['sigmaY0'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaZ0'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaqPt0'], {"colorZvar":"pidCode"}],
    #
    [['qPt'], ['sigmaY0Rot'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaZ0Rot'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaqPt0Rot'], {"colorZvar":"pidCode"}],
    
    #
    [['qPt'], ['sigmaY0/sqrt(1+qPt**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaZ0/sqrt(1+qPt**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaqPt0/sqrt(1+qPt**2)'], {"colorZvar":"pidCode"}],
    #
    [['qPt'], ['sigmaY0/sqrt(1+(dEdxExp*qPt)**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaZ0/sqrt(1+(dEdxExp*qPt)**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaqPt0/sqrt(1+(dEdxExp*qPt)**2)'], {"colorZvar":"pidCode"}],
    #
    ['table'],
    {"size": 5}
]

widgetParams=[
    ['range', ['qPt']],
    ['range', ['tglMC']],
    #
    ['range', ['sigmaY0']],
    ['range', ['sigmaZ0']],
    ['range', ['sigmaqPt0']],
    #
    ['multiSelect', ['pidCode']],
    ['multiSelect', ["isSecondary"]],
    #
    ['range', ['fMaxLayer']],
    ['range', ['snpMax']],
     ['range', ['dEdxExp']],
    
]
tooltips = [("qPt","@qPt")]
widgetLayoutDesc=[ [0,1],[2,3,4],[5,6],[7,8,9], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0,1,2, {'plot_height':170}],
    [3,4,5, {'plot_height':170}],
    [6,7,8, {'plot_height':170}],
    [9,10,11, {'plot_height':170}],
    {'plot_height':400,'sizing_mode':'scale_width'}
]

output_file("fastSimulDisplayResolution.html")
figC=bokehDrawSA.fromArray(dfDraw, "fMaxLayer>4", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=20000,rescaleColorMapper=True,arrayCompression=arrayCompressionRelative8)

In [ ]:
dfDraw.head(5)

In [ ]:
tree.Draw("gyMC",defaultCutTrack)
c1.Draw()

In [ ]:
defaultCutTrack="ptMC>0.01&&abs(tglMC)<1&&abs(gzMC)<300&&abs(gyMC)<300"
#dfResDraw=tree2Panda(tree,["gyMC","qPt","tglMC","sigmaY0","sigmaZ0","sigmaqPt0","pidCode","isSecondary","fMaxLayer","snpMax","dEdxExp"],
#                 "",columnMask=[["_fElements",""]],nEntries=5000)
dfResDraw=tree2Panda(tree,["gyMC","gxMC","gyIn","gxIn","gzIn","gzMC","ptMC","ptMCP","tglMC","pidCode","qPt","trackID","pIndex","isSecondary","sigmaY0","sigmaZ0","sigmaqPt0","dEdxExp","fMaxLayer"],
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=2000)
dfResDraw.head(5)

In [ ]:


output_file("fastSimulDisplayReolutionBugTracing.html")
dfDraw=dfResDraw.head(40000)

histoArray = [
    {"name": "histogdX", "variables": ["gxMC-gxIn"], "nbins": 50},
    {"name": "histogdY", "variables": ["gyMC-gyIn"], "nbins": 50},
    {"name": "histogdZ", "variables": ["gzMC-gzIn"], "nbins": 50}
]

figureArray = [
    [['qPt'], ['sigmaY0'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaZ0'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaqPt0'], {"colorZvar":"pidCode"}],
    #
    [['qPt'], ['sigmaY0/sqrt(1+qPt**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaZ0/sqrt(1+qPt**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaqPt0/sqrt(1+qPt**2)'], {"colorZvar":"pidCode"}],
    #
    [['qPt'], ['sigmaY0/sqrt(1+(dEdxExp*qPt)**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaZ0/sqrt(1+(dEdxExp*qPt)**2)'], {"colorZvar":"pidCode"}],
    [['qPt'], ['sigmaqPt0/sqrt(1+(dEdxExp*qPt)**2)'], {"colorZvar":"pidCode"}],
    #
    [['gxMC'], ['gyMC'], {"colorZvar":"qPt"}],
    [['sqrt(gxMC**2+gyMC**2)'], ['gzMC'], {"colorZvar":"qPt"}],
    [['sqrt(gxMC**2+gyMC**2)'], ['gyMC-gyIn'], {"colorZvar":"qPt"}],
    
    #
    ['table'],
    {"size": 5}
]

widgetParams=[
    ['range', ['qPt']],
    ['range', ['tglMC']],
    #
    ['range', ['sigmaY0']],
    ['range', ['sigmaZ0']],
    ['range', ['sigmaqPt0']],
    #
    ['range', ['pidCode']],
    ['select', ["isSecondary"]],
    #
    ['range', ['fMaxLayer']],
#    ['range', ['snpMax']],
     ['range', ['dEdxExp']],
     ['range', ['gyIn']],
    
]
tooltips = [("qPt","@qPt")]
widgetLayoutDesc=[ [0,1],[2,3,4],[5,6],[7,8], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0,1,2, {'plot_height':200}],
    [3,4,5, {'plot_height':200}],
    [6,7,8, {'plot_height':200}],
    [9,10,11, {'plot_height':200}],
    {'plot_height':400,'sizing_mode':'scale_width'}
]

output_file("fastSimulDisplayResolutionDebug.html")
figC=bokehDrawSA.fromArray(dfDraw, "pidCode>-1", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=5000,rescaleColorMapper=True,arrayCompression=arrayCompressionRelative8)

## Dashboard resolution qPt, pidCode
* at low pt and high dEdx outliers
* is it rotation poroblem ?
  * max snp along track added

## 2.) XYZ display pt 
<a id='XYZdisplayPt'></a>

In [ ]:
defaultCutTrack="ptMC>0.01&&abs(tglMC)<1&&abs(gzMC)<300&&abs(gyMC)<300"
dfTrackDraw=tree2Panda(tree,["gyMC","gxMC","rMC","gyIn","gxIn","gzIn","gzMC","ptMC","ptMCP","tglMC","pidCode","qPt","trackID","pIndex","isSecondary"],
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=2000)

In [ ]:
dfTrackDraw.head(5)

In [ ]:
tree.SetAlias("trackID","Entry$")
tree.SetAlias("gzMC","part.fParamMC[].fP[1]");
defaultCutTrack="ptMC>0.01&&abs(tglMC)<1&&abs(gzMC)<300&&abs(gyMC)<300"
dfTrackDraw=tree2Panda(tree,["gyMC","gxMC","rMC","gyIn","gxIn","gzIn","gzMC","ptMC","ptMCP","tglMC","pidCode","qPt","trackID","pIndex","isSecondary"],
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=5000)
output_file("fastSimulDisplay.html")
dfDraw=dfTrackDraw.head(400000)
figureArray = [
    [['gxMC'], ['gyMC'], {"colorZvar":"qPt"}],
    [['rMC'], ['gzMC'], {"colorZvar":"qPt"}],
    [['gxMC'], ['gyMC'], {"colorZvar":"qPt", "varZ": "gzMC","dot-size":1}],
    ['table'],
    {"size": 2}
]

widgetParams=[
    ['range', ['gyMC']],
    ['range', ['gxMC']],
    ['range', ['qPt']],
    ['range', ['pidCode']],
    ['range', ['trackID']],
    ['range', ['pIndex']],
    ['select', ["isSecondary"]]
]
tooltips = [("gx", "@gxMC"), ("gy", "@gyMC"), ("pidCode", "@pidCode"), ("pt", "@ptMC"), ("pt point", "@ptMCP"),("trackID","@trackID"), ("pIndex","@pIndex")]
widgetLayoutDesc=[ [0,1],[2,3],[4,5,6], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0,1, 2,{'plot_height':500}],
    {'plot_height':400,'sizing_mode':'scale_width'}
]
#fig=bokehDrawSA.fromArray(dfDraw, "pidCode>-1", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=5000,rescaleColorMapper=True)
output_file("fastSimulDisplayComp.html")
figC=bokehDrawSA.fromArray(dfDraw, "pidCode>-1", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=5000,rescaleColorMapper=True,arrayCompression=arrayCompressionRelative8)

In [ ]:
output_file("fastSimulDisplayHisto.html")
tree.SetAlias("trackID","Entry$")
tree.SetAlias("gzMC","part.fParamMC[].fP[1]");
defaultCutTrack="ptMC>0.0&&abs(tglMC)<2&&abs(gzMC)<300&&abs(gyMC)<300"
dfTrackDraw=tree2Panda(tree,["gyMC","gxMC","gyIn","gxIn","gzIn","gzMC","ptMC","tglMC","pidCode","qPt","trackID","isSecondary","rMC","snpMC","pIndex"],
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=200)
output_file("fastSimulDisplayTrack.html")
dfDraw=dfTrackDraw.head(50000)

histoArray = [
    {"name": "histogX", "variables": ["gxMC"], "nbins": 50},
    {"name": "histogY", "variables": ["gyMC"], "nbins": 50},
    {"name": "histogXY", "variables": ["gxMC", "gyMC"], "nbins": [40, 40]},
]

figureArray = [
    [['gxMC'], ['gyMC',"gyIn"], {"colorZvar":"trackID"}],
    [['sqrt(gxMC**2+gyMC**2)'], ['gzMC'], {"colorZvar":"trackID"}],
    [['gxIn'], ['gyIn'], {"colorZvar":"trackID", "varZ": "gzMC","dot-size":1}],
    [['gxMC'], ['histogX'], {}],
    [['gyMC'], ['histogY'], {}],
    [['gyMC'], ['histogXY'], {}],
    ['table'],
    {"size": 4}
]

widgetParams=[
    ['range', ['gyMC']],
    ['range', ['gxMC']],
    ['range', ['rMC']],
    ['range', ['snpMC']],
    ['range', ['qPt']],
    ['select', ['pidCode']],
    ['range', ['trackID']],
    ['range', ['pIndex']],
    ['select', ['isSecondary']]
]
tooltips = [("gx", "@gxMC"), ("gy", "@gyMC"), ("pidCode", "@pidCode"), ("pt", "@ptMC"),("trackID","@trackID"),("pIndex","@pIndex")]
widgetLayoutDesc=[ [0,1],[2,3],[4,5],[6,7], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0,1,2, {'plot_height':300}],
    [3,4,5, {'plot_height':300}],
    {'plot_height':400,'sizing_mode':'scale_width'}
]
fig=bokehDrawSA.fromArray(dfDraw, "pidCode>-1", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=5000,rescaleColorMapper=True,histogramArray=histoArray)
output_file("fastSimulDisplayHistoComp.html")
fig=bokehDrawSA.fromArray(dfDraw, "pidCode>-1", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=5000,rescaleColorMapper=True,histogramArray=histoArray,arrayCompression=arrayCompressionRelative8)

In [ ]:
from pprint import *
tree.SetAlias("trackID","Entry$")
tree.SetAlias("gzMC","part.fParamMC[].fP[1]");
defaultCutTrack="ptMC>0.0&&abs(tglMC)<2&&abs(gzMC)<300&&abs(gyMC)<300"
dfTrackDraw=tree2Panda(tree,["gyMC","gxMC","gyIn","gxIn","gzIn","gzMC","ptMC","tglMC","pidCode","qPt","trackID","isSecondary","rMC","snpMC","pIndex"],
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=200)
pprint(dfTrackDraw.columns)
dfTrackDraw["dxIn"]=dfTrackDraw.eval("gxMC-gxIn")
dfTrackDraw["dyIn"]=dfTrackDraw.eval("gyMC-gyIn")
dfTrackDraw["dzIn"]=dfTrackDraw.eval("gzMC-gzIn")

In [ ]:
output_file("fastSimulDisplayHisto.html")
dfDraw=dfTrackDraw.head(50000)

histoArray = [
    {"name": "histogX", "variables": ["gxMC"], "nbins": 50},
    {"name": "histogY", "variables": ["gyMC"], "nbins": 50},
    {"name": "histogZ", "variables": ["gzMC"], "nbins": 50},
    #
    {"name": "histodX", "variables": ["dxIn"], "nbins": 50},
    {"name": "histodY", "variables": ["dyIn"], "nbins": 50},
    {"name": "histodZ", "variables": ["dzIn"], "nbins": 50},
]

figureArray = [
    [['gxMC'], ['gyMC',"gyIn"], {"colorZvar":"trackID"}],
    [['sqrt(gxMC**2+gyMC**2)'], ['gzMC'], {"colorZvar":"trackID"}],
    [['gxIn'], ['gyIn'], {"colorZvar":"trackID", "varZ": "gzMC","dot-size":1}],
    #
    [['gxMC'], ['histogX'], {}],
    [['gyMC'], ['histogY'], {}],
    [['gzMC'], ['histogZ'], {}],
    #
    [['gxMC'], ['histodX'], {}],
    [['gyMC'], ['histodY'], {}],
    [['gzMC'], ['histodZ'], {}],
    ['table'],
    {"size": 4}
]

widgetParams=[
    ['range', ['gyMC']],
    ['range', ['gxMC']],
    ['range', ['rMC']],
    ['range', ['snpMC']],
    ['range', ['qPt']],
    ['select', ['pidCode']],
    ['range', ['trackID']],
    ['range', ['pIndex']],
    ['select', ['isSecondary']],
    ['range', ['dxIn']],
    ['range', ['dyIn']],
    ['range', ['dzIn']],
]
tooltips = [("gx", "@gxMC"), ("gy", "@gyMC"), ("pidCode", "@pidCode"), ("pt", "@ptMC"),("trackID","@trackID"),("pIndex","@pIndex")]
widgetLayoutDesc=[ [0,1,2,3],[4,5, 6,7,8], [9,10,11], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0,1,2, {'plot_height':200}],
    [3,4,5, {'plot_height':200}],
    [6,7,8, {'plot_height':200}],
    {'plot_height':200,'sizing_mode':'scale_width'}
]
fig=bokehDrawSA.fromArray(dfDraw, "pidCode>-1", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=5000,rescaleColorMapper=True,histogramArray=histoArray)
output_file("fastSimulDisplayHistoComp.html")
fig=bokehDrawSA.fromArray(dfDraw, "pidCode>-1", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=5000,rescaleColorMapper=True,histogramArray=histoArray,arrayCompression=arrayCompressionRelative8)

# Fast prototypeng  part 

* check treee content to define interactive plots

In [ ]:
tree.GetListOfAliases().Print("","*g*In*")

In [ ]:
c = ROOT.TCanvas("xxx","xxx")
tree.Draw("gyMC:gxMC:Iteration$","Iteration$<100","colz",1,31)
c.Draw("")
#tree.Scan("gyMC:gxMC:Iteration$","Iteration$<100","",1,31)

In [ ]:
c = ROOT.TCanvas("xxx","xxx")
tree.Draw("sigmaY0:qPt:pidCode","fMaxLayer==159&&abs(part.fParamMC[fMaxLayer].fP[2])<0.90&&abs(sigmaY0)<1","colz")
c.Draw("")

In [ ]:
# Check status
tree.Draw("gyMC:gxMC:Iteration$","Iteration$<100","colz",1,31)
c.Draw("")